In [110]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

In [111]:
# Google Mobility data
global_data = pd.read_csv("global.csv", low_memory=False)

In [112]:
global_select = global_data.loc[(global_data["country_region"] == "Austria") | (global_data["country_region"] == "Belgium") |
                (global_data["country_region"] == "Bulgaria") | (global_data["country_region"] == "Croatia") |
                (global_data["country_region"] == "Czech Republic") | (global_data["country_region"] == "Denmark") |
                (global_data["country_region"] == "France") | (global_data["country_region"] == "Germany") |
                (global_data["country_region"] == "Greece") | (global_data["country_region"] == "Hungary") |
                (global_data["country_region"] == "Ireland") | (global_data["country_region"] == "Italy") |
                (global_data["country_region"] == "Latvia") | (global_data["country_region"] == "Lithuania") |
                (global_data["country_region"] == "Luxembourg") | (global_data["country_region"] == "Malta") |
                (global_data["country_region"] == "Netherlands") | (global_data["country_region"] == "Poland") |
                (global_data["country_region"] == "Portugal") | (global_data["country_region"] == "Romania") |
                (global_data["country_region"] == "Slovakia") | (global_data["country_region"] == "Slovenia") |
                (global_data["country_region"] == "Spain") | (global_data["country_region"] == "Sweden") |
                (global_data["country_region"] == "United Kingdom")]

In [113]:
google_data = global_select.loc[global_select["sub_region_1"].isnull()]
google_data_m = google_data.rename(columns = {"country_region": "region", 
                                              "retail_and_recreation_percent_change_from_baseline": "g_retail_recreation",
                                              "grocery_and_pharmacy_percent_change_from_baseline": "g_grocery_pharmacy",
                                              "parks_percent_change_from_baseline": "g_parks",
                                              "transit_stations_percent_change_from_baseline": "g_transit_stations",
                                              "workplaces_percent_change_from_baseline": "g_workplaces",
                                              "residential_percent_change_from_baseline": "g_residential"})
google_data_s = google_data_m[["date", "region", "g_retail_recreation", "g_grocery_pharmacy",
                              "g_parks", "g_transit_stations", "g_workplaces", "g_residential"]]
google_data_r = google_data_s.set_index(["date", "region"])

In [114]:
# Apple mobility data
apple_data = pd.read_csv("apple_mob.csv")
apple_data_r = apple_data.set_index(["date", "region"])

In [115]:
# Airline data
airline_data = pd.read_csv("airline_cleaned_new.csv")
airline_data['date'] = airline_data['date'].str.replace('-','')
airline_data["date"] = pd.to_datetime(airline_data["date"], format = '%Y%m%d')

In [116]:
airline_data_r = airline_data.set_index(['date', 'region'])

In [117]:
# merge the data sets together
merge1 = pd.merge(google_data_r, apple_data_r, on = ["date", "region"])

In [118]:
merge1_r = merge1.reset_index()
merge1_r['date']=pd.to_datetime(merge1_r['date'])
merge1_r2 = merge1_r.set_index(["date", "region"])

In [120]:
merge2 = pd.merge(merge1_r2, airline_data_r, on = ["date", "region"])
merge_data = merge2.reset_index()
merge_data

,date,region,g_retail_recreation,g_grocery_pharmacy,g_parks,g_transit_stations,g_workplaces,g_residential,apple_driving,apple_transit,apple_walking,departure,arrival,total_flt,departure_p,arrival_p,total_p
0,2020-02-15,Austria,9.0,1.0,42.0,13.0,0.0,-2.0,158.950000,NaN,165.310000,533,536,1069,0.038986,0.011321,0.024928
1,2020-02-16,Austria,15.0,21.0,42.0,12.0,1.0,-2.0,120.220000,NaN,113.750000,496,468,964,-0.060606,-0.141284,-0.101584
2,2020-02-17,Austria,9.0,5.0,35.0,3.0,-4.0,0.0,113.560000,NaN,125.350000,486,495,981,-0.022133,0.083151,0.028302
3,2020-02-18,Austria,8.0,5.0,40.0,2.0,-4.0,0.0,113.490000,NaN,127.470000,443,429,872,-0.022075,-0.075431,-0.049073
4,2020-02-19,Austria,4.0,2.0,10.0,-1.0,-5.0,1.0,117.980000,NaN,125.250000,473,473,946,0.145278,0.170792,0.157895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8087,2021-01-27,Slovakia,-59.0,-18.0,-10.0,-52.0,-38.0,16.0,66.930000,54.419998,78.320000,7,8,15,-0.666667,-0.466667,-0.583333
8088,2021-01-28,Slovakia,-59.0,-16.0,-18.0,-53.0,-37.0,16.0,67.160004,53.119999,79.070000,11,9,20,-0.560000,-0.625000,-0.591837
8089,2021-01-29,Slovakia,-61.0,-14.0,-25.0,-54.0,-36.0,17.0,69.330002,51.810001,76.209999,8,10,18,-0.600000,-0.545455,-0.571429
8090,2021-01-30,Slovakia,-64.0,-17.0,-21.0,-52.0,-16.0,9.0,53.060001,49.490002,64.139999,3,4,7,-0.875000,-0.840000,-0.857143


In [14]:
# Facebook data
# not used
fb_data = pd.read_csv('facebook.txt', delimiter = "\t")

/Users/cyl/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
fb_data

,ds,country,polygon_source,polygon_id,polygon_name,all_day_bing_tiles_visited_relative_change,all_day_ratio_single_tile_users,baseline_name,baseline_type
0,2020-03-01,AGO,GADM,AGO.10.10_1,Lubango,-0.02992,0.18751,full_february,DAY_OF_WEEK
1,2020-03-02,AGO,GADM,AGO.10.10_1,Lubango,0.06746,0.10521,full_february,DAY_OF_WEEK
2,2020-03-03,AGO,GADM,AGO.10.10_1,Lubango,0.05873,0.11397,full_february,DAY_OF_WEEK
3,2020-03-04,AGO,GADM,AGO.10.10_1,Lubango,0.01288,0.10492,full_february,DAY_OF_WEEK
4,2020-03-05,AGO,GADM,AGO.10.10_1,Lubango,0.02753,0.11056,full_february,DAY_OF_WEEK
...,...,...,...,...,...,...,...,...,...
5913773,2020-12-12,ZWE,GADM,ZWE.9.1_1,Beitbridge,0.08277,0.09513,full_february,DAY_OF_WEEK
5913774,2020-12-13,ZWE,GADM,ZWE.9.1_1,Beitbridge,0.03378,0.09936,full_february,DAY_OF_WEEK
5913775,2020-12-17,ZWE,GADM,ZWE.9.1_1,Beitbridge,-0.00025,0.08457,full_february,DAY_OF_WEEK
5913776,2020-12-18,ZWE,GADM,ZWE.9.1_1,Beitbridge,-0.05873,0.08148,full_february,DAY_OF_WEEK
